# Sentiment Abstracts

In this notebook, we investigate the sentiment present in article abstracts.

## 1. Libraries

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
import tqdm
import swifter

## 2. Loading data

In [3]:
df = pd.read_parquet(r"..\Data\cleaned_parquet\final\P5_final_new.parquet")
df.head()

uid                                              title  \
0  10186596  The potential impact of health care reform on ...   
1  10186588  New Jersey health promotion and disease preven...   
2  10186587  Who will provide preventive services? The chan...   
3  10163501  Cytoreduction of small intestine metastases us...   
4  10157383  Racial differences in access to kidney transpl...   

                                             journal  \
0  Journal of public health management and practi...   
1  Journal of public health management and practi...   
2  Journal of public health management and practi...   
3                     Journal of gynecologic surgery   
4                       Health care financing review   

                                            abstract  \
0  General: This article observes that, despite t...   
1  General: Health promotion is a major component...   
2  General: Health care reform in the United Stat...   
3  General: The Cavitron Ultrasonic Surgical Aspi...   
4  General: Previous work has documented large di...   

                             authors  \
0              Auerbach J; McGuire J   
1                         Louria D B   
2  Pearson T A; Spencer M; Jenkins P   
3                        Adelson M D   
4                         Eggers P W   

                                        affiliations  \
0  HIV/AIDS Bureau, Massachusetts Department of P...   
1  Department of Preventive Medicine and Communit...   
2  Mary Imogene Bassett Research Institute, Coope...   
3  Department of Obstetrics and Gynecology, Crous...   
4  Office of Research and Demonstrations, Health ...   

                                          mesh_terms  \
0  Financing, Government; HIV Infections; Health ...   
1  Female; Health Education; Health Promotion; Hu...   
2  Delivery of Health Care; Female; Health Care R...   
3  Adenocarcinoma; Fallopian Tube Neoplasms; Fema...   
4  Adolescent; Adult; Black or African American; ...   

                                            keywords coi_statement  \
0                                                              N/A   
1                                                              N/A   
2                                                              N/A   
3                                                              N/A   
4  Empirical Approach; End Stage Renal Disease Pr...           N/A   

  parsed_date  ...                            cleaned_title_tokens_hf  \
0  1995-01-01  ...  [[CLS], potential, impact, health, care, refor...   
1  1995-01-01  ...  [[CLS], new, jersey, health, promotion, diseas...   
2  1995-01-01  ...  [[CLS], provide, prevent, ##ive, services, ?, ...   
3  1995-01-01  ...  [[CLS], cy, ##tore, ##duction, small, int, ##e...   
4  1995-01-01  ...  [[CLS], racial, differences, access, kidney, t...   

                      cleaned_abstract_tokens_simple  \
0  [general, article, observes, despite, clear, p...   
1  [general, health, promotion, major, component,...   
2  [general, health, care, reform, united, states...   
3  [general, cavitron, ultrasonic, surgical, aspi...   
4  [general, previous, work, documented, large, d...   

                          cleaned_abstract_tokens_hf  \
0  [[CLS], general, article, observes, despite, c...   
1  [[CLS], general, health, promotion, major, com...   
2  [[CLS], general, health, care, reform, united,...   
3  [[CLS], general, ca, ##vi, ##tron, ultra, ##so...   
4  [[CLS], general, previous, work, documented, l...   

  disease_title_tokens_simple disease_title_tokens_hf  \
0                       [hiv]                   [hiv]   
1                          []                      []   
2                          []                      []   
3                          []                      []   
4                          []                      []   

  disease_abstract_tokens_simple disease_abstract_tokens_hf  \
0                    [hiv, aids]                [hiv, aids]   
1                  

In [10]:
df.columns

Index(['uid', 'title', 'journal', 'abstract', 'authors', 'affiliations',
       'mesh_terms', 'keywords', 'coi_statement', 'parsed_date',
       'cleaned_title_tokens_simple', 'cleaned_title_tokens_hf',
       'cleaned_abstract_tokens_simple', 'cleaned_abstract_tokens_hf',
       'disease_title_tokens_simple', 'disease_title_tokens_hf',
       'disease_abstract_tokens_simple', 'disease_abstract_tokens_hf',
       'disease_abstract_spacy', 'disease_title_spacy',
       'disease_mesh_terms_spacy'],
      dtype='object')

## 3. Sentiment Analysis (VADER) on abstracts

In [14]:
sia = SentimentIntensityAnalyzer()
df[['neg_vader', 'neu_vader', 'pos_vader', 'compound_vader']] = df['abstract'].swifter.apply(
    lambda text: pd.Series(sia.polarity_scores(str(text)))
)

Pandas Apply:   0%|          | 0/1057871 [00:00<?, ?it/s]

### 3.1. Average score

In [18]:
mean_scores = df[['neg_vader', 'neu_vader', 'pos_vader', 'compound_vader']].mean()

print("Average sentiment scores:")
print(mean_scores)

Average sentiment scores:
neg_vader         0.059812
neu_vader         0.874477
pos_vader         0.065708
compound_vader    0.082011
dtype: float64


We can observe that on average abstracts, as expected, are neutral.

### 3.2. Sentiment Trends Over Time

In [19]:
sentiment_per_year = df.groupby(df['parsed_date'].dt.year)[['neg_vader', 'neu_vader', 'pos_vader', 'compound_vader']].mean()

sentiment_per_year.plot(kind='line', figsize=(12, 6), marker='o', title='Sentiment Trends Over Time')
plt.xlabel('Year')
plt.ylabel('Average Sentiment Score')
plt.legend(title="Sentiment Type")
plt.show()

We can observe that sentiment in abstracts does not change significantly over time. There is only a slight decrease in negative sentiment from 2010 onwards.

### 3.3. Monthly Sentiment Trends

In [ ]:
compound_vader_per_month_avg = df.groupby(df['parsed_date'].dt.month)['compound_vader'].mean()

plt.figure(figsize=(12, 6))
compound_vader_per_month_avg.plot(kind='bar', title='Average Compound Vader Per Month')
plt.xlabel('Month')
plt.ylabel('Average Compound Vader')
plt.xticks(ticks=range(0, 12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=0)
plt.show()

We can observe that any month in a year is not associated with negative sentiment in abstracts. For all of the months, sentiment stays more or less the same.

## 4. Conclusions

1. On average abstracts, as expected, are neutral.

2. Sentiment in abstracts does not change significantly over time. There is only a slight decrease in negative sentiment from 2010 onwards.

3. Any month in a year is not associated with negative sentiment in abstracts. For all of the months, sentiment stays more or less the same.